In [1]:
!pip install opencv-python pillow > /dev/null 2>&1

! mkdir img_folder > /dev/null 2>&1
! mkdir label_folder > /dev/null 2>&1
! mkdir grayscale_folder > /dev/null 2>&1
! mkdir overlay_folder > /dev/null 2>&1

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os
import cv2
import numpy as np
import json
from PIL import Image, ImageDraw

def folder_with_labels_to_grayscale(img_folder, label_folder, grayscale_folder, overlay_folder, label_to_index, color_palette):
    os.makedirs(grayscale_folder, exist_ok=True)
    os.makedirs(overlay_folder, exist_ok=True)

    for label_file in os.listdir(label_folder):
        if not label_file.endswith(".json"):
            continue

        base_name = os.path.splitext(label_file)[0]
        image_filename = base_name + ".png"
        image_path = os.path.join(img_folder, image_filename)
        label_path = os.path.join(label_folder, label_file)

        if not os.path.exists(image_path):
            print(f"Image {image_filename} not found. Skipping.")
            continue

        with open(label_path, 'r') as f:
            data = json.load(f)

        # Load image to get dimensions
        image = cv2.imread(image_path)
        height, width = image.shape[:2]

        # Create empty mask (grayscale)
        mask = Image.new("L", (width, height), 255)  # 255 = unlabeled
        draw = ImageDraw.Draw(mask)

        for shape in data['shapes']:
            label_name = shape['label']
            if label_name not in label_to_index:
                continue

            polygon = shape['points']
            class_index = label_to_index[label_name]

            draw.polygon(polygon, fill=class_index)

        # Convert mask to NumPy array
        mask_np = np.array(mask)

        # Generate color mask
        color_mask = np.zeros((height, width, 3), dtype=np.uint8)
        for class_index, color in color_palette.items():
            color_mask[mask_np == class_index] = color

        # Convert input image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray_image_rgb = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)

        # Save grayscale image
        grayscale_path = os.path.join(grayscale_folder, image_filename)
        cv2.imwrite(grayscale_path, mask_np)

        # Overlay mask on grayscale
        overlay = cv2.addWeighted(gray_image_rgb, 0.6, color_mask, 0.4, 0)

        # Save overlay
        overlay_path = os.path.join(overlay_folder, image_filename)
        cv2.imwrite(overlay_path, overlay)

        print(f"Finished: {base_name}")

    print(f"Grayscale images saved in: {grayscale_folder}")
    print(f"Overlay images saved in: {overlay_folder}")


# Guarda los jsons en label_folder y las imagenes en img_folder. Corre todos los codigos y mira el resultado en overlay_folder



In [10]:
img_folder = '/content/drive/MyDrive/Facultad/Onceavo semestre/TSCF/Dataset/Diciembre/imgs'
label_folder = '/content/drive/MyDrive/Facultad/Onceavo semestre/TSCF/Dataset/Diciembre/LabelMe'
grayscale_folder = '/content/grayscale_folder'
overlay_folder = '/content/overlay_folder'


label_to_index = {
    'road': 0,
    'vegetation': 1,
    'sky': 2,
    'obstacle': 3,
    'other': 4
}

color_palette = {
    0: (128, 64, 128),
    1: (0, 128, 0),
    2: (70, 130, 180),
    3: (220, 20, 60),
    4: (128, 128, 128)
}

folder_with_labels_to_grayscale(
    img_folder,
    label_folder,
    grayscale_folder,
    overlay_folder,
    label_to_index,
    color_palette
)

Finished: 221205_0045
Finished: 221205_0014
Finished: 221205_0001
Finished: 221205_0002
Finished: 221205_0010
Finished: 221205_0012
Finished: 221205_0013
Finished: 221205_0015
Finished: 221205_0016
Finished: 221205_0017
Finished: 221205_0018
Finished: 221205_0019
Finished: 221205_0020
Finished: 221205_0022
Finished: 221205_0023
Finished: 221205_0024
Finished: 221205_0025
Finished: 221205_0026
Finished: 221205_0027
Finished: 221205_0028
Finished: 221205_0029
Finished: 221205_0030
Finished: 221205_0031
Finished: 221205_0032
Finished: 221205_0034
Finished: 221205_0036
Finished: 221205_0037
Finished: 221205_0038
Finished: 221205_0039
Finished: 221205_0040
Finished: 221205_0041
Finished: 221205_0044
Finished: 221205_0048
Finished: 221205_0000
Finished: 221205_0003
Finished: 221205_0005
Finished: 221205_0007


KeyboardInterrupt: 

In [ ]:
!cp -r /content/grayscale_folder/ "/content/drive/MyDrive/Facultad/Onceavo semestre/TSCF/Dataset/AgroPipe/annotations"
!cp -r "/content/drive/MyDrive/Facultad/Onceavo semestre/TSCF/Dataset/Diciembre/imgs/" "/content/drive/MyDrive/Facultad/Onceavo semestre/TSCF/Dataset/AgroPipe/imgs"

In [ ]:
!rm -rf /content/drive/MyDrive/FACULTAD/TSCF/OurDataset/grayscale_folder